In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x
!pip install -q keras-vggface
!pip install -q scikit-image
!pip install -q pydot
!pip install -q keras==2.2.5
!pip install h5py==2.10.0

TensorFlow 1.x selected.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline

import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
import skimage
from skimage.transform import rescale, resize

import pydot

Using TensorFlow backend.


In [ ]:
print(tf.__version__)
print(keras.__version__)

1.15.2
2.2.5


In [ ]:
EPOCHS = 50
BS = 64
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 170

ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

Resize_pixelsize = 197

In [ ]:
vgg_notop = VGGFace(model='resnet50', include_top=False, input_shape=(Resize_pixelsize, Resize_pixelsize, 3), pooling='avg')
last_layer = vgg_notop.get_layer('avg_pool').output

x = Flatten(name='flatten')(last_layer)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc7')(x)
x = Dropout(DROPOUT_RATE)(x)
    
batch_norm_indices = [2, 6, 9, 13, 14, 18, 21, 24, 28, 31, 34, 38, 41, 45, 46, 53, 56, 60, 63, 66, 70, 73, 76, 80, 83, 87, 88, 92, 95, 98, 102, 105, 108, 112, 115, 118, 122, 125, 128, 132, 135, 138, 142, 145, 149, 150, 154, 157, 160, 164, 167, 170]
for i in range(FROZEN_LAYER_NUM):
    if i not in batch_norm_indices:
        vgg_notop.layers[i].trainable = False

out = Dense(7, activation='softmax', name='classifier')(x)

model = Model(vgg_notop.input, out)


optim = keras.optimizers.Adam(lr=ADAM_LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = keras.optimizers.SGD(lr=SGD_LEARNING_RATE, momentum=0.9, decay=SGD_DECAY, nesterov=True)
rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_acc',mode='max',factor=0.5, patience=4, min_lr=0.00001, verbose=1)

mfile = '/content/drive/MyDrive/aml/saved/resnet-cw/my_best_model.epoch{epoch:02d}-loss{val_loss:.3f}.hdf5'
msave = keras.callbacks.ModelCheckpoint(filepath=mfile,verbose=1, save_weights_only=True,period= 10 )

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
print(model)














94699520/94694792 [==============================] - 1s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [ ]:
! rm -rf train; mkdir train
! unzip -q '/content/drive/MyDrive/aml/datasets/fer2013/train.zip' -d train

! rm -rf dev; mkdir dev
! unzip -q '/content/drive/MyDrive/aml/datasets/fer2013/test-public.zip' -d dev

! rm -rf test; mkdir test
! unzip -q '/content/drive/MyDrive/aml/datasets/fer2013/test-private.zip' -d test

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datagen(dataset, aug=False):
    if aug:
        datagen = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(
            dataset,
            target_size=(197, 197),
            color_mode='rgb',
            shuffle = True,
            class_mode='categorical',
            batch_size=BS)

In [ ]:
train_generator  = get_datagen('/content/train', True)
dev_generator    = get_datagen('/content/dev')
test_generator  = get_datagen('/content/test')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [ ]:
from sklearn.utils import class_weight
file_stream = file_io.FileIO('/content/drive/MyDrive/aml/datasets/fer2013/dev.csv', mode='r')
data = pd.read_csv(file_stream)
data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])
X, Y = data['pixels'].tolist(), data['emotion'].values
class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(Y),
                                        y = Y                                                    
                                    )

In [ ]:
class_weights

array([1.09788926, 9.15561224, 1.03369816, 0.57286512, 0.78516736,
       1.23545611, 0.84466933])

In [ ]:
history = model.fit_generator(
    generator = train_generator,
    validation_data=dev_generator, 
    steps_per_epoch=28709// BS,
    validation_steps=3589 // BS,
    shuffle=True,
    epochs=100,
    callbacks=[rlrop, msave],
    class_weight=class_weights
) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
448/448 [==============================] - 320s 714ms/step - loss: 2.4910 - acc: 0.3827 - val_loss: 1.2456 - val_acc: 0.5491
Epoch 2/100
448/448 [==============================] - 307s 685ms/step - loss: 1.3273 - acc: 0.5025 - val_loss: 1.1311 - val_acc: 0.5782
Epoch 3/100
448/448 [==============================] - 307s 685ms/step - loss: 1.2217 - acc: 0.5443 - val_loss: 1.1084 - val_acc: 0.5918
Epoch 4/100
448/448 [==============================] - 306s 683ms/step - loss: 1.1629 - acc: 0.5663 - val_loss: 1.0752 - val_acc: 0.6065
Epoch 5/100
448/448 [==============================] - 305s 682ms/step - loss: 1.1222 - acc: 0.5802 - val_loss: 1.0299 - val_acc: 0.6298
Epoch 6/100
448/448 [==============================] - 306s 682ms/step - loss: 1.0881 - acc: 0.5959 - val_loss: 1.0195 - val_acc: 0.6216
Epoch 7/100
448/448 [==============================] - 306s 682ms/step - loss: 1.0

In [ ]:
model.load_weights('/content/drive/MyDrive/aml/saved/resnet-cw/my_best_model.epoch30-loss0.855.hdf5')

In [ ]:
history = model.fit_generator(
    generator = train_generator,
    validation_data=dev_generator, 
    steps_per_epoch=28709// BS,
    validation_steps=3589 // BS,
    shuffle=True,
    epochs=70,
    callbacks=[rlrop, msave],
    class_weight=class_weights
) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/70
448/448 [==============================] - 471s 1s/step - loss: 0.8207 - acc: 0.6922 - val_loss: 0.8927 - val_acc: 0.6830
Epoch 2/70
448/448 [==============================] - 454s 1s/step - loss: 0.8185 - acc: 0.6907 - val_loss: 0.8971 - val_acc: 0.6871
Epoch 3/70
448/448 [==============================] - 453s 1s/step - loss: 0.8059 - acc: 0.6967 - val_loss: 0.9100 - val_acc: 0.6794
Epoch 4/70
448/448 [==============================] - 453s 1s/step - loss: 0.7964 - acc: 0.7005 - val_loss: 0.8906 - val_acc: 0.6933
Epoch 5/70
448/448 [==============================] - 453s 1s/step - loss: 0.7921 - acc: 0.7020 - val_loss: 0.8774 - val_acc: 0.6877
Epoch 6/70
448/448 [==============================] - 453s 1s/step - loss: 0.7687 - acc: 0.7108 - val_loss: 0.8706 - val_acc: 0.6965
Epoch 7/70
448/448 [==============================] - 453s 1s/step - loss: 0.7633 - acc: 0.7132 - val_loss

In [ ]:
model.load_weights('/content/drive/MyDrive/aml/saved/resnet-cw/my_best_model.epoch70-loss0.872.hdf5')

In [ ]:
history = model.fit_generator(
    generator = train_generator,
    validation_data=dev_generator, 
    steps_per_epoch=28709// BS,
    validation_steps=3589 // BS,
    shuffle=True,
    epochs=30,
    callbacks=[rlrop, msave],
    class_weight=class_weights
) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
448/448 [==============================] - 488s 1s/step - loss: 0.5572 - acc: 0.7877 - val_loss: 0.8515 - val_acc: 0.7146
Epoch 2/30
448/448 [==============================] - 471s 1s/step - loss: 0.5751 - acc: 0.7850 - val_loss: 0.8711 - val_acc: 0.6967
Epoch 3/30
448/448 [==============================] - 471s 1s/step - loss: 0.5801 - acc: 0.7831 - val_loss: 0.8634 - val_acc: 0.7095
Epoch 4/30
448/448 [==============================] - 470s 1s/step - loss: 0.5611 - acc: 0.7917 - val_loss: 0.8679 - val_acc: 0.7055
Epoch 5/30
448/448 [==============================] - 471s 1s/step - loss: 0.5590 - acc: 0.7909 - val_loss: 0.8845 - val_acc: 0.7021

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
Epoch 6/30
448/448 [==============================] - 470s 1s/step - loss: 0.5245 - acc: 0.8035 - val_loss: 0.8647 - val_acc: 0.7081
Epoch 7/30
448/448 [======

In [ ]:
model.load_weights("/content/drive/MyDrive/aml/models/RESNET50-CW-EPOCHS_100_test_acc_0.72.hdf5")

In [ ]:
results_dev = model.evaluate_generator(dev_generator, 3589 // BS)
print('dev loss:', results_dev[0],", dev acc:", results_dev[1])

dev loss: 0.8762221772755895 , dev acc: 0.7154017857142857


In [ ]:
results_test = model.evaluate_generator(test_generator, 3599 // BS)
print('test loss:', results_test[0],", test acc:", results_test[1])

test loss: 0.7861265039869717 , test acc: 0.7310267857142857


In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
epoch_str = '-EPOCHS_' + str(100)
test_acc = 'test_acc_%.3f' % results_test[1]
model.save('/content/drive/MyDrive/aml/models/' + 'RESNET50-CW' + epoch_str + test_acc + '.h5')